In [1]:
import pandas as pd
import numpy as np
import pickle
import treecorr
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
N=500

In [3]:
df_riz = pd.read_pickle('/home/cz136/project/balrog/data/mcal{}_df_all.pkl'.format(3)) 

In [4]:
length=len(df_riz)

In [5]:
keys = df_riz.columns[40:]
# keys=['ebv_planck13']

In [6]:
true_df=pd.read_pickle("/home/cz136/project/balrog/data/true_df.pkl")
df_riz_list= [df_riz]

In [7]:
def add_tilename(df_list,true):
    for i in range(4):
        df_list[i]=pd.concat([df_list[i],true_df[['meas_tilename']]],axis=1,join='inner')

In [8]:
df_riz=pd.concat([df_riz,true_df[['meas_tilename']]],axis=1,join='inner')

In [9]:
from scipy.stats import truncnorm

def get_truncated_normal(mean=0, sd=0.24, low=-1, upp=1):

    return truncnorm(
        (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)

In [10]:
#foo_data

# df_riz['e_1']=generator.rvs(length)
# df_riz['e_2']=generator.rvs(length)
# df_riz['true_cm_g_1']=generator.rvs(length)
# df_riz['true_cm_g_2']=generator.rvs(length)


In [11]:
# for key in keys:
#     df_riz[key]=np.ones(length)

In [12]:
def get_ke(catalog,key):
    
    
    delta_g1=catalog['e_1'].to_numpy()
    delta_g2=catalog['e_2'].to_numpy()
    k=catalog[key].to_numpy()
    ra=catalog['ra'].to_numpy()
    dec=catalog['dec'].to_numpy()

    
    kcat= treecorr.Catalog( k=k,
                                 ra=ra, dec=dec, 
                                 ra_units='deg', dec_units='deg')
    
    gcat = treecorr.Catalog( g1=delta_g1, g2=delta_g2,
                                 ra=ra, dec=dec, 
                                 ra_units='deg', dec_units='deg')
    
    kg = treecorr.KGCorrelation(nbins=20, min_sep=2.5, max_sep=250, sep_units='arcmin', bin_slop=0.2)
    kg.process(kcat,gcat)
    return (kg.xi)

In [13]:
def get_ke_true(catalog,key):

    
    
    
    delta_g1=catalog['true_cm_g_1'].to_numpy()
    delta_g2=catalog['true_cm_g_2'].to_numpy()
    k=catalog[key].to_numpy()
    ra=catalog['ra'].to_numpy()
    dec=catalog['dec'].to_numpy()
    
    kcat= treecorr.Catalog( k=k,
                                 ra=ra, dec=dec, 
                                 ra_units='deg', dec_units='deg')
    
    gcat = treecorr.Catalog( g1=delta_g1, g2=delta_g2,
                                 ra=ra, dec=dec, 
                                 ra_units='deg', dec_units='deg')
    
    kg = treecorr.KGCorrelation(nbins=20, min_sep=2.5, max_sep=250, sep_units='arcmin', bin_slop=0.2)
    kg.process(kcat,gcat)
    
    return (kg.xi)

In [14]:
def get_chisqr(covariance,y_value):
    return(np.dot(np.dot(y_value,np.linalg.inv(covariance)),y_value))

def get_cov(jk_samples_for_key):
    
    xi_array=np.array(jk_samples_for_key[0])
    xi_true_array=np.array(jk_samples_for_key[1])
    
    diff_array=xi_array-xi_true_array
    
    n = len(diff_array)
    
    emp_average=(1/n)*np.sum(diff_array,axis=0)
    nd = len(diff_array[0])
    cov=np.empty([nd,nd])
    for row in range(nd):
        for column in range(nd):
            cov[row][column]=\
            1/n * np.sum((diff_array[:,row]-emp_average[row])*(diff_array[:,column]-emp_average[column]))
    return (cov)

def get_y_value(key):
    y_value_true = get_ke(df_riz,key)
    y_value = get_ke_true(df_riz,key)
    diff=y_value-y_value_true
    return (diff)

def get_x_value(catalog,key):
        
    delta_g1=catalog['e_1'].to_numpy()
    delta_g2=catalog['e_2'].to_numpy()
    k=catalog[key].to_numpy()
    ra=catalog['ra'].to_numpy()
    dec=catalog['dec'].to_numpy()

    
    kcat= treecorr.Catalog( k=k,
                                 ra=ra, dec=dec, 
                                 ra_units='deg', dec_units='deg')
    
    gcat = treecorr.Catalog( g1=delta_g1, g2=delta_g2,
                                 ra=ra, dec=dec, 
                                 ra_units='deg', dec_units='deg')
    
    kg = treecorr.KGCorrelation(nbins=20, min_sep=2.5, max_sep=250, sep_units='arcmin', bin_slop=0.2)
    kg.process(kcat,gcat)
    return (np.exp(kg.meanlogr))

In [15]:
def get_bs_resample(df,n):
    bs_index_list=[]
    for i in tqdm(range(n)):
        random_index = np.random.choice(df.index.to_numpy(),size=length,
                                       replace=True)
        bs_index_list.append(random_index)
    return (bs_index_list)

bs_index_list=get_bs_resample(df_riz,N)

100%|██████████| 300/300 [00:03<00:00, 82.72it/s]


In [16]:
def get_jk_ke_resample(df,bs_index_list,key):
    
    n=len(bs_index_list)
        
    xi_matrix=np.empty([n,20])
    xi_true_matrix=np.empty([n,20])
    
    for i,bs_index in tqdm(enumerate(bs_index_list)):
        
        bs=df.loc[bs_index]
        
        xi_matrix[i,:]=get_ke(bs,key)
        xi_true_matrix[i,:]=get_ke_true(bs,key)
        
        np.save("/home/cz136/project/balrog/data/bootstrap/{}".format(key),xi_matrix)
        np.save("/home/cz136/project/balrog/data/bootstrap/{}_true".format(key),xi_true_matrix)
 
    return(xi_matrix,xi_true_matrix)

In [17]:
def get_chi_sqr_for_key(jk_ke_resample,key):
    print("Getting covariance")
    cov = get_cov(jk_ke_resample)
    y_value=get_y_value(key)
)

In [18]:
def get_chi_sqr_from_df(df,key):
 jk_ke_resample=get_jk_ke_resample(df_riz,bs_index_list,key)
    return(get_chi_sqr_for_key(jk_ke_resample,key))

In [19]:
chi_sqr_array=np.empty(len(keys))

with open('/home/cz136/project/balrog/data/chi_sqr_bootstrap.txt', 'w+') as f:
    for i,key in enumerate(keys):
        print (key)
        chi_sqr=get_chi_sqr_from_df(df_riz,key)
        chi_sqr_array[i]=chi_sqr
        print(chi_sqr)
        f.write("{}={}".format(key,chi_sqr))
        
np.save("/home/cz136/project/balrog/data/bootstrap/chi_sqr_bs".format(key),chi_sqr_array)

0it [00:00, ?it/s]

ebv_planck13


KeyboardInterrupt: 

In [20]:
# chi_matrix={}
# chi_true_matrix={}
# for key in keys[:5]:
#     chi_matrix[key]=np.load("/home/cz136/project/balrog/data/bootstrap/{}.npy".format(key))
#     chi_true_matrix[key]=np.load("/home/cz136/project/balrog/data/bootstrap/{}_true.npy".format(key))

In [21]:
# test_key=keys[0]

In [22]:
# test_y=get_y_value(test_key)
# test_x=get_x_value(df_riz,test_key)

In [23]:
# import matplotlib.pyplot as plt
# fig, ax = plt.subplots()
# ax.plot(test_x,test_y)
# plt.show()